In [1]:
import numpy as np
import pandas as pd
import collections
import time
import graphviz
import pydotplus
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from dateutil import parser, rrule
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error

In [2]:
# Read the csv data file.
trips = pd.read_csv('Bixi_data/OD_2018-06.csv')

# Shape[0] gives the number of rows in the array
tripsNumb = trips.shape[0]
print(tripsNumb)

888470


In [3]:
# Get the day, month, and year of the trips
trips.start_date = pd.to_datetime(trips.start_date) - pd.to_timedelta(pd.to_datetime(trips.start_date).dt.second, unit='s')
trips.end_date = pd.to_datetime(trips.end_date) - pd.to_timedelta(pd.to_datetime(trips.end_date).dt.second, unit='s')

# pd.to_datetime will change the argument (trips.start/end_date) from the csv file into datetime
# pd.to_timedelta will give the absolute differences in time of the arguement

trips['month'] = trips.start_date.dt.month
trips['weekday'] = trips.start_date.dt.weekday
trips['hour'] = trips.start_date.dt.hour
trips['round_to_hour'] = trips.start_date.dt.round("H")
trips['date'] = trips.start_date.dt.date

In [4]:
# Read the hourly temperature csv file of the corresponding month
temperature = pd.read_csv('Weather_data/en_climate_hourly_QC_702S006_06-2018_P1H.csv')

# Make a dataframe of date and time with the temperature
tempDataFrame = pd.concat([temperature['Date/Time'], temperature['Temp (°C)']], axis=1)
tempDataFrame['Date/Time'] = pd.to_datetime(tempDataFrame['Date/Time'])
tempDataFrame = tempDataFrame.set_index('Date/Time').T

# .T gives the transpose of the dataframe

# Make the dataframe a dictionary
tempDict = tempDataFrame.to_dict('list')

In [5]:
# Read the daily temperature and precipitation csv file of the corresponding year
ppt = pd.read_csv('Weather_data/en_climate_daily_QC_702S006_2018_P1D.csv')

# Make a dataframe of date and time with the precipitation data, then make it a dictionary
rainDataFrame = pd.concat([ppt['Date/Time'], ppt['Total Precip (mm)']], axis=1)
rainDataFrame['Date/Time'] = pd.to_datetime(rainDataFrame['Date/Time'])
rainDataFrame['Date/Time'] = rainDataFrame['Date/Time'].dt.date
rainDataFrame = rainDataFrame.fillna(0)
rainDataFrame = rainDataFrame.set_index('Date/Time').T
rainDict = rainDataFrame.to_dict('list')

# .fillna(0) will replace empty slost with zero

# Make a dataframe of date and time with the snow data, then make it a dictionary
snowDataFrame = pd.concat([ppt['Date/Time'], ppt['Total Snow (cm)']], axis=1)
snowDataFrame['Date/Time'] = pd.to_datetime(snowDataFrame['Date/Time'])
snowDataFrame['Date/Time'] = snowDataFrame['Date/Time'].dt.date
snowDataFrame = snowDataFrame.fillna(0)
snowDataFrame = snowDataFrame.set_index('Date/Time').T
snowDict = snowDataFrame.to_dict('list')

# Map the temperature, rain, and snow dictionaries to the trips
trips['Temp'] = trips.round_to_hour.map(tempDict)
trips['Rain'] = trips.date.map(rainDict)
trips['Snow'] = trips.date.map(snowDict)

# Make the respective object columns of type int
trips['Temp'] = (trips.Temp.str[0])
trips['Temp'] = pd.to_numeric(trips.Temp , errors='ignore')
trips['Temp'] = trips.Temp.fillna(0)
trips['Temp'] = trips.Temp.astype(int)

trips['Rain'] = (trips.Rain.str[0])
trips['Rain'] = pd.to_numeric(trips.Rain, errors='ignore')
trips['Rain'] = trips.Rain.fillna(0)
trips['Rain'] = trips.Rain.astype(int)

trips['Snow'] = (trips.Snow.str[0])
trips['Snow'] = pd.to_numeric(trips.Snow, errors='ignore')
trips['Snow'] = trips.Snow.fillna(0)
trips['Snow'] = trips.Snow.astype(int)

In [6]:
# Calculate amount of trips and number of bikes used, then make a dictionary of the time and amount
firstTrip = min(trips.start_date)
lastTRip = max(trips.end_date)
amountTime = list(rrule.rrule(freq = rrule.MINUTELY, dtstart = firstTrip, until = lastTRip))
amountValue = np.zeros(len(amountTime))

# .zeros creates an empty list

amountDict = dict(zip(amountTime,amountValue))
for i in np.arange(tripsNumb):
    start = trips.start_date.iloc[i]
    end = trips.end_date.iloc[i]
    for j in list(rrule.rrule(freq = rrule.MINUTELY, dtstart = start, until = end)):
        amountDict[j] += 1

trips['amount'] = trips.start_date.map(amountDict)
amount = trips.amount.values

In [7]:
# Natural Data
naturalDataFrame = pd.concat([trips['hour'], trips['weekday'], trips['month'], trips['Temp'],trips['Rain'], trips['Snow'], trips['amount']], axis = 1)
xNaturalDataFrame = naturalDataFrame.drop(['amount'], axis = 1)
xMatrixNatural = xNaturalDataFrame.astype(float).values

In [8]:
# Engineered Data
engineeredDataFrame = naturalDataFrame

# Set the weekdays to 1 and weekends to 0
# Weekdays have indexes 0 to 4, weekends have indexes 5 and 6
engineeredDataFrame.weekday = engineeredDataFrame.weekday.replace(np.arange(5), 1)
engineeredDataFrame.weekday = engineeredDataFrame.weekday.replace(5,0)
engineeredDataFrame.weekday = engineeredDataFrame.weekday.replace(6,0)

# Set rain/snow to 1 if it is raining/snowing
engineeredDataFrame.loc[engineeredDataFrame.Rain > 0] = 1
engineeredDataFrame.loc[engineeredDataFrame.Snow > 0] = 1

# Set the cold months (like January, February..) to 0 and the warmer months (June, July..) to 1
engineeredDataFrame.month = engineeredDataFrame.month.replace(np.arange(0,4), 0)
engineeredDataFrame.month = engineeredDataFrame.month.replace(4, 0)
engineeredDataFrame.month = engineeredDataFrame.month.replace(np.arange(5,10), 1)
engineeredDataFrame.month = engineeredDataFrame.month.replace(np.arange(10,13), 0)

xEngineeredDataFrame = engineeredDataFrame.drop(['amount'], axis = 1)
xMatrixEngineered = xEngineeredDataFrame.astype(float).values

In [9]:
# Make training and testing dataset, use 70% of the data for training and 30% for testing
np.random.seed(2)
train = int(tripsNumb*0.7)
trainSet = np.random.choice(tripsNumb, train, replace=False)
testSet = np.array(list(set(range(tripsNumb))-set(trainSet)))
testNumb = tripsNumb - train

# Training
XTrainNat = xMatrixNatural[trainSet, :]
XTrainEngineered = xMatrixEngineered[trainSet, :]
YTrain = amount[trainSet]

# Testing
XTestNat = xMatrixNatural[testSet, :]
XTestEngineered = xMatrixEngineered[testSet, :]
YTest = amount[testSet]

In [11]:
# Plotting the decision tree
def treePlot(fit):
    labels = ['Temperature (C)', 'Total Rain (mm)', 'Total Snow (cm)', 'Hour', 'Weekday', 'Month']

    dotData = tree.export_graphviz(fit, out_file=None,max_depth=3, filled=True,rounded=True,special_characters=True,
                                   feature_names = labels, impurity=False)

# To have different colors in the graph
    graph = pydotplus.graph_from_dot_data(dotData)
    colors = ('thistle1', 'violet')
    edges = collections.defaultdict(list)
    for edge in graph.get_edge_list():
        edges[edge.get_source()].append(int(edge.get_destination()))
    for edge in edges:
        edges[edge].sort()
        for i in range(2):
            dest = graph.get_node(str(edges[edge][i]))[0]
            dest.set_fillcolor(colors[i])
    return graph

In [14]:
# Minimum number of sample at a leaf node
parameters = {'min_samples_leaf': [10, 50, 100, 250, 500, 750, 1000, 1250, 1500, 1750, 2000]}

# Training the natural data
decisionTreeNatural = DecisionTreeRegressor()
fitNatural = GridSearchCV(decisionTreeNatural, parameters, cv=5, refit = True)
fitNatural.fit(XTrainNat, YTrain)
treePredictNat = fitNatural.predict(XTestNat)

# best_params_ : which parameter gives best results
bestParamNat = fitNatural.best_params_
print(bestParamNat)  # = 10

# Testing the natural data
treeNatural = DecisionTreeRegressor(min_samples_leaf= 10)
treeNatural.fit(XTrainNat, YTrain)
treePredictNat =  treeNatural.predict(XTestNat)

graph = treePlot(treeNatural)
graph.write_png('NaturalTree.png')

# Setting squared to false returns root mean squared error, instead of mean squared error
rmse = mean_squared_error(treePredictNat, YTest, squared = False)
percentError = mean_absolute_percentage_error(treePredictNat, YTest)

print("RMSE of the natural data = " , rmse)
print("Mean Absolute Percent Error of the natural data = " , percentError)

{'min_samples_leaf': 10}
RMSE of Natural Data =  58.373723701726874
Mean Absolute Percent Error of Natural Data =  0.08250636940680331


In [17]:
# Training the engineered data

decisionTreeEngineered = DecisionTreeRegressor()
fitEngineered = GridSearchCV(decisionTreeEngineered, parameters, cv=5, refit = True)
fitEngineered.fit(XTrainEngineered, YTrain)
treePredictEngineered = fitEngineered.predict(XTestEngineered)
bestParamEngineered = fitEngineered.best_params_
print(bestParamEngineered)  # = 10

# Testing the engineered data
treeEngineered = DecisionTreeRegressor(min_samples_leaf = 10)
treeEngineered.fit(XTrainEngineered, YTrain)
treePredictEngineered = treeEngineered.predict(XTestEngineered)

graph = treePlot(treeEngineered)
graph.write_png('EngineeredTree.png')

rmse = mean_squared_error(treePredictEngineered, YTest, squared = False)
percentError = mean_absolute_percentage_error(treePredictEngineered, YTest)

print("RMSE of the engineered data = " , rmse)
print("Mean Absolute Percent Error of the natural data = " , percentError)

{'min_samples_leaf': 10}
RMSE of the engineered data =  137.1051864164655
Mean Absolute Percent Error of the natural data =  0.19480058666602337
